<a href="https://colab.research.google.com/github/NITHIN-KESHAV/DATA-228-Group-Project/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gdown


## Working on chunk data( 10,000 rows )

In [0]:
import gdown

# The updated file ID from your Google Drive link
file_id = "11CHzQfUChpnG1EGRWQXPs4VYIcmtVQZj"
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
output = 'yourfile.zip'
gdown.download(download_url, output, quiet=False)



In [0]:
import os

# Check the size of the file to ensure it was downloaded completely
file_size = os.path.getsize('yourfile.zip')
print(f"File size: {file_size / (1024 * 1024):.2f} MB")


In [0]:
# Check the first few bytes of the file to see if it's a valid zip file
with open('yourfile.zip', 'rb') as file:
    header = file.read(4)
    print(header)


In [0]:
with gzip.open('yourfile.zip', 'rb') as f:
    chunk_size = 100000  # Adjust the chunk size as needed
    for chunk in pd.read_csv(f, chunksize=chunk_size, sep='\t'):  # Change sep if it's not a CSV
        print(chunk.head())
        break



In [0]:
with gzip.open('yourfile.zip', 'rb') as f:
    for i, line in enumerate(f):
        print(line)
        if i == 20:  # Stop after the first 20 lines
            break



In [0]:
import gzip
import pandas as pd

# Open the gzip file and read the data as JSON lines
with gzip.open('yourfile.zip', 'rb') as f:
    # Read the JSON lines into a pandas DataFrame
    df = pd.read_json(f, lines=True)

# Display the first few rows to get a better view of the data
print(df.head(5))


In [0]:
import gzip
import pandas as pd

# Define the chunk size (number of lines to process at a time)
chunk_size = 10000  # You can adjust this value

# Open the gzip file and process it in chunks
with gzip.open('yourfile.zip', 'rb') as f:
    # Read the file in chunks and process each chunk
    for chunk in pd.read_json(f, lines=True, chunksize=chunk_size):
        print(chunk.head())  # Display the first few rows of each chunk

        # Add your processing logic here, e.g., cleaning, analysis, etc.

        # Stop after the first chunk for testing (remove this in your final implementation)
        break


In [0]:
print(chunk.columns)  # List the column names
print(chunk.dtypes)   # Check the data types of each column


In [0]:
print(chunk.describe())  # Summary statistics for numerical columns


In [0]:
print(chunk.isnull().sum())  # Check for missing values in each column


In [0]:
chunk['review_length'] = chunk['reviewText'].apply(lambda x: len(str(x).split()))
print(chunk[['reviewText', 'review_length']].head())  # Show the first few rows with review lengths


In [0]:
from google.colab import files

# Download the saved file
files.download('processed_chunk.csv')


## steps for topic modelling for chunks


In [0]:
!pip install gensim scikit-learn


In [0]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/processed_chunk.csv')

# Check the data structure
print(df.head())


## Preprocess the Text for Topic Modeling:

In [0]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import nltk

# Download stopwords if not done already
nltk.download('stopwords')

# Preprocessing function: clean and tokenize text
def preprocess_text(text):
    text = re.sub(r'\W', ' ', str(text))  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    tokens = text.lower().split()  # Lowercase and tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return tokens

# Apply preprocessing to the 'reviewText' column
df['tokens'] = df['reviewText'].apply(preprocess_text)

# Check the tokenized reviews
print(df[['reviewText', 'tokens']].head())


##  Create Dictionary and Corpus:

In [0]:
from gensim import corpora

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary(df['tokens'])

# Create a bag-of-words corpus
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# Show sample corpus
print(corpus[:1])


## Apply LDA for Topic Modeling:

In [0]:
import gensim

# Set the number of topics
num_topics = 10

# Run LDA using gensim
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                            num_topics=num_topics,
                                            id2word=dictionary,
                                            passes=10)

# Print the topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)


### Visualize the Topics

In [0]:
!pip install pyLDAvis


In [0]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Prepare the visualization data
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)

# Display the visualization
pyLDAvis.display(vis_data)
